In [2]:
from data_preparation import gff_to_csvs, gff_to_csv
from data_dir import (annotated_grch38_gff, annotated_grch38_gff_dir, annotated_grch38_gff_csv)

print(annotated_grch38_gff)
print(annotated_grch38_gff_csv)
print(annotated_grch38_gff_dir)

./data/genome/grch38/GRCh38_latest_genomic.gff
./data/genome/grch38/grch38_gff.csv
./data/genome/grch38/csvs


In [1]:
from data_dir import chr1_index_csv, annotated_grch38_gff_dir, chr1_fasta, data_genome_grch38_labels_dir
from data_preparation import kmer
import os 
from tqdm import tqdm

from data_dir import chr1_fasta
from Bio import SeqIO

records = SeqIO.parse(chr1_fasta, 'fasta')
genome_record = next(records)
genome_str = str(genome_record.seq)
genome_length = len(genome_str)
annotation = ['.' for i in range(genome_length)]

f = open(chr1_index_csv, 'r')
next(f) # Skip first line since first line is just header.


for line in tqdm(f, total=352637):
    """
    Since the file is basically CSV, parsing CSV is carried out manually line by line.
    The header is this.
    `sequence_id,refseq,region,start_index,end_index,start,end,gene,gene_id,genbank,ensembl`
    Enjoy tah siah!
    """
    arr_line = line.strip().split(',')
    seq_id = arr_line[0]
    refseq = arr_line[1]
    region = arr_line[2]
    start_index = int(arr_line[3])
    end_index = int(arr_line[4])
    start = int(arr_line[5])
    end = int(arr_line[6])
    
    if region == 'region':
        """
        This is the genome range. `annotation` has been initialized based on sequence in genome fasta file.
        Compare the size and use the largest.
        """
        if len(annotation) < end:
            annotation = ['.' for i in range(end)]
        
    elif region == 'exon':
        """
        This is exon. Take the region and write 'E' on annotation sequence.
        """
        start_region = start_index
        end_region = end
        for j in range(start_index, end):
            annotation[j] = 'E'
    else:
        continue
#endfor
fin_annotation = ''.join(annotation)
"""
Compare the sequence and the annotation sequence. 
If annotation sequence is smaller than sequence, pad it with '.'.
If DNA sequence is smaller, pad it with 'N'.
"""
annotation_length = len(annotation)
delta = genome_length - annotation_length
if (delta > 0):
    print('padding annotation')
    for k in range(0, delta):
        annotation.append('.')
if (delta < 0):
    print('padding sequence')
    for k in range(0, delta):
        genome_str = genome_str + 'N'

annotation_str = ''.join(annotation)
arr_annotation_str = kmer(annotation_str, 512, 512)
arr_genome_str = kmer(genome_str, 512, 512)
path = '{}/chr1_annotation.csv'.format(data_genome_grch38_labels_dir)
if os.path.exists(path):
    os.remove(path)
g = open(path, 'x')
g.write('sequence,label\n')
for seq, label in zip(arr_genome_str, arr_annotation_str):
    g.write('{},{}\n'.format(seq, label))
g.close()

100%|██████████| 352637/352637 [00:04<00:00, 78798.55it/s]


In [19]:
s = 'asgagdsff'
s = s + ('XXX')
s

'asgagdsffXXX'

In [15]:
genome_str[248956421:248956422]

'N'

In [1]:
from data_dir import (
    chr1_index_csv, chr2_index_csv, chr3_index_csv, chr4_index_csv, chr5_index_csv, chr6_index_csv, chr7_index_csv, chr8_index_csv, chr9_index_csv, chr10_index_csv,
    chr11_index_csv, chr12_index_csv, chr13_index_csv, chr14_index_csv, chr15_index_csv, chr16_index_csv, chr17_index_csv, chr18_index_csv, chr19_index_csv, chr20_index_csv,
    chr21_index_csv, chr22_index_csv, chr23_index_csv, chr24_index_csv
)
from data_dir import (
    chr1_fasta, chr2_fasta, chr3_fasta, chr4_fasta, chr5_fasta, chr6_fasta, chr7_fasta, chr8_fasta, chr9_fasta, chr10_fasta, 
	chr11_fasta, chr12_fasta, chr13_fasta, chr14_fasta, chr15_fasta, chr16_fasta, chr17_fasta, chr18_fasta, chr19_fasta, chr20_fasta, 
	chr21_fasta, chr22_fasta, chr23_fasta, chr24_fasta,
)
chr_fastas = [
    chr1_fasta,
	#chr2_fasta, chr3_fasta, chr4_fasta, chr5_fasta, chr6_fasta, chr7_fasta, chr8_fasta, chr9_fasta, chr10_fasta,
	#chr11_fasta, chr12_fasta, chr13_fasta, chr14_fasta, chr15_fasta, chr16_fasta, chr17_fasta, chr18_fasta, chr19_fasta, chr20_fasta,
	#chr21_fasta, chr22_fasta, chr23_fasta, chr24_fasta
]
from data_dir import labseq_dir, labseq_names
from data_preparation import generate_sequence_labelling
chr_indices = [
    chr1_index_csv, 
	#chr2_index_csv, chr3_index_csv, chr4_index_csv, chr5_index_csv, chr6_index_csv, chr7_index_csv, chr8_index_csv, chr9_index_csv, chr10_index_csv,
    #chr11_index_csv, chr12_index_csv, chr13_index_csv, chr14_index_csv, chr15_index_csv, chr16_index_csv, chr17_index_csv, chr18_index_csv, chr19_index_csv, chr20_index_csv,
    #chr21_index_csv, chr22_index_csv, chr23_index_csv, chr24_index_csv
]
chr_labseq_path = ["{}/{}".format(labseq_dir, fname) for fname in labseq_names[0:1]]
for src, fasta, target in zip(chr_indices, chr_fastas, chr_labseq_path):
    print("Generating sequential labelling for index {}, from fasta {}, to {}: {}".format(src, fasta, target, generate_sequence_labelling(src, fasta, target, do_expand=True, expand_size=512)))

MemoryError: 

In [2]:
from sequential_labelling import Label_Dictionary
import pandas as pd
from tqdm import tqdm
import os

def prepare_sequence_from_csv(src_csv, label_dictionary=Label_Dictionary):
    """
    Convert sequence into tokenized DNA sequence and label sequence.
    CSV source has columns `sequence` and `label`. 
    `sequence` contains tokenized DNA sequence and `label` contains sequence of labels.
    @param      src_csv (string): path to CSV source.
    @param      label_dictionary (dict): dictionary to convert label into number.
    @return     sequence, labels
    """
    if not os.path.exists(src_csv):
        raise FileNotFoundError(src_csv)
    if label_dictionary == None:
        raise Exception("Argument `label_dictionary` cannot be empty!")
    
    df = pd.read_csv(src_csv)
    return list(df['sequence']), list(df['label'])
    

['a', 'a', 'a', 'a', 'a', 'b', 'a', 'a', 'a', 'a']